In [21]:
import cv2
import mediapipe as mp
import numpy as np
import time

#mediapipe객체 생성
mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose

#비디오 캡쳐
video_path="jec.mp4"
cap = cv2.VideoCapture("jec2.mp4")  # 웹캠 캡처
cap_video=cv2.VideoCapture(video_path)  # 비디오 파일 캡처

# 선택된 랜드마크 리스트
SELECTED_LANDMARKS = [
    'NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
    'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW',
    'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
    'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE',
    'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX'
]

#1~33까지 인덱스 중에서 위에서 고른 랜드마크의 고유 인덱스를 얻어서 S_L_I 변수에 저장
SELECTED_LANDMARK_INDICES = [mp_pose.PoseLandmark[landmark].value for landmark in SELECTED_LANDMARKS]

#관절의 각도를 계산해주는 함수
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)

#관절의 각도 계산 b관절을 기준으로 a관절과 c관절의 cos값을 계산. cos값이 -1~1까지 나오므로 코사인의 역함수 값을 취하면 0~180도가 나옴. cos(-1): 180 cos(0):90
#cos (0도) = 1   cos(90도) =0 cos (180도) = -1

def process_frame(frame, pose):
    frame.flags.writeable = False #프레임을 읽기 모드로 염
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # cv2를 활용해서 BGR을 RGB형식으로 변환(pose는 RGB형식 이미지를 사용)
    results = pose.process(frame) #pose 객체로 프레임을 처리해 관절의 좌표획득
    frame.flags.writeable = True #프레임을 쓰기 모드로 염
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) #cv2로 보여주기위해 이미지를 BGR형식으로 다시 바꿔줌

    if results.pose_landmarks: #랜드마크가 있다면
        landmarks = results.pose_landmarks.landmark #감지된 랜드마크를 landmarks 변수에 할당
        connections_to_draw = [
            connection for connection in mp_pose.POSE_CONNECTIONS # MediaPipe에서 정의한 포즈 랜드마크 간의 기본 연결 목록을 돌면서 connection로 읽어들임
            if connection[0] in SELECTED_LANDMARK_INDICES and connection[1] in SELECTED_LANDMARK_INDICES
        ] #랜드 마크 고유 인덱스를 돌면서 랜드마크 연결목록이 둘다 랜드마크 인덱스에 있다면 connections_to_draw 리스트에 추가
        
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, connections_to_draw,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )#이미지 프레임 위에 landmarks를 그림
        
# results.pose_landmarks:랜드마크 객체
# connections_to_draw:선택된 랜드마크 간의 연결 리스트
# mp_drawing.DrawingSpec: 연결할때 스타일 지정 첫번째는 랜드마크 점의 스타일 두번째는 선의 스타일

    return frame, results

#관절의 좌표값을 얻어서 angles리스트에 담아 cal angles함수로 보낸 후 나온 각도 결과값을 angles에 순서대로 저장
def get_angles(landmarks):
    angles = []
    
    # 오른쪽 팔꿈치 각도
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
    angles.append(angle)

    # 왼쪽 팔꿈치 각도
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    angles.append(angle)
    
    # 오른쪽 무릎 각도
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    angle = calculate_angle(right_hip, right_knee, right_ankle)
    angles.append(angle)

    # 왼쪽 무릎 각도
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    angle = calculate_angle(left_hip, left_knee, left_ankle)
    angles.append(angle)

    # 오른쪽 발목 각도
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
    angle = calculate_angle(right_knee, right_ankle, right_foot_index)
    angles.append(angle)

    # 왼쪽 발목 각도
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                       landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
    angle = calculate_angle(left_knee, left_ankle, left_foot_index)
    angles.append(angle)

    # 오른쪽 골반 각도
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    angle = calculate_angle(right_shoulder, right_hip, right_knee)
    angles.append(angle)

    # 왼쪽 골반 각도
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    angle = calculate_angle(left_shoulder, left_hip, left_knee)
    angles.append(angle)

    return angles

# 각도 차이 지속 시간을 추적하기 위한 타이머 추가
angle_difference_timers = [0] * 8  # 8개의 관절에 대한 타이머
angle_difference_start_times = [None] * 8  # 각 관절의 각도 차이 시작 시간
last_print_times = [0] * 8  # 각 관절의 마지막 출력 시간을 저장하는 변수

# 메인 루프 부분
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose_webcam, \
     mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose_video: 
    #min_detection_confidence와 min_tracking_confidence는 각각 최소 감지 신뢰도와 최소 추적 신뢰도를 설정하고 Pose객체를 초기화
    while cap.isOpened() and cap_video.isOpened():#웹캠의 cap이 열려있는동안 반복실행
        ret, frame = cap.read()#cap의 프레임이 제대로 들어갔는지에 대한 불리언값(ret)과 그 프레임의 이미지(frame)를 저장
        ret_video, frame_video = cap_video.read() #비디오에 대해서도 실행

        if not ret or not ret_video: #ret이나 ret_video에 False가 들어가면 프린트 즉 영상이 끝나거나 웹캠이 꺼졌다면 break
            print("Can't receive frame (stream end?). Exiting ...")
            break

        frame, results = process_frame(frame, pose_webcam) #이미지에 대해 process_frame를 적용해서 frame(랜드마크가 그려진 이미지)과 results(관절 좌표)를 얻음
        frame_video, results_video = process_frame(frame_video, pose_video)#비디오이미지에 대해 process_frame를 적용해서 frame(랜드마크가 그려진 이미지)과 results(관절 좌표)를 얻음

        if results.pose_landmarks and results_video.pose_landmarks: #두 이미지의 results에 랜드마크가 있으면
            angles_webcam = get_angles(results.pose_landmarks.landmark) #웹캠의 각도를 얻음
            angles_video = get_angles(results_video.pose_landmarks.landmark)#비디오의 각도를 얻음
            #print(angles_webcam)
            #print(angles_video)
            angle_differences = np.array(angles_webcam) - np.array(angles_video) #웹캠과 비디오의 차이를 얻음
            
            # 각도 차이를 화면에 표시
            joint_names = ["Right Elbow", "Left Elbow", "Right Knee", "Left Knee", 
               "Right Ankle", "Left Ankle", "Right Hip", "Left Hip"]

            current_time = time.time()
            

            for i, (name, diff) in enumerate(zip(joint_names, angle_differences)):
                if abs(diff) >= 15:  # 각도 차이가 15도 이상일 때
                    if angle_difference_start_times[i] is None:  # 새로운 각도 차이 시작
                        angle_difference_start_times[i] = current_time  # 시작 시간 기록
                    
                    angle_difference_timers[i] = current_time - angle_difference_start_times[i]
                    
                    if angle_difference_timers[i] >= 4.0 and (current_time - last_print_times[i]) >= 5:  # 4초 이상 지속되고, 마지막 출력으로부터 5초 이상 지났을 때
                        #message = f"{name} 각도차이: {diff:.2f} 도 ({angle_difference_timers[i]:.2f}초 지속됨)"
                        #print(message)
                        cv2.putText(frame, message, 
                                    (10, 30 + i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                        last_print_times[i] = current_time
                        print(f"All angle differences:{angle_differences}")
                else:
                    angle_difference_timers[i] = 0  # 각도 차이가 15도 미만이면 타이머 리셋
                    angle_difference_start_times[i] = None  # 시작 시간 초기화

                # 모든 각도 차이를 항상 화면에 표시
                cv2.putText(frame, f"{name}: {diff:.2f}", 
                            (10, frame.shape[0] - 30 - i*30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

            
            # 각도 차이 배열 출력 (15도 이상 차이가 4초 이상 지속된 경우에만, 그리고 마지막 출력으로부터 5초 이상 지났을 때)
            if should_print_angles and (current_time - last_print_time) >= 5:
                print(f"All angle differences: {angle_differences}")
                last_print_time = current_time

        # 이미지 크기 맞추기
        height = min(frame.shape[0], frame_video.shape[0])
        width = min(frame.shape[1], frame_video.shape[1])

        frame_resized = cv2.resize(frame, (width, height))
        frame_video_resized = cv2.resize(frame_video, (width, height))

        # 두 영상을 나란히 표시
        combined_frame = np.hstack((frame_resized, frame_video_resized))
        cv2.imshow('Webcam and Video Comparison', combined_frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cap_video.release()
cv2.destroyAllWindows()

#같은 영상으로 관절 각도의 차이를 봤을때 이론상 0이 나와야 맞지만 개개인의 키와 특성을 고려했을때 아래 수치는 오차값이 작은 수준이므로 위의 각도 계산식으로 운동법 피드백을 위한 참고 자료로 사용가능하다고 판단.
#openpose와 비교했을 때 mediapipe의 실시간 추적이 더 빠르기때문에 실시간 웹캠과 비교하기에 더 적합하다고 판단하여 mediapipe사용
#openpose는 bottom-up방식으로 인물이 늘어나도 비슷한 계산시간을 소요하지만
#mediapipe는 top-down방식으로 인물 객체를 먼저 찾고 관절로 내려가기 떄문에 인물수에 따라 계산 시간이 선형적으로 증가

#angle differences = 웹캠비디오의 해당 관절의 각도 - 운동비디오의 해당관절의 각도
#angle differences 값이 양수이면 웹캠의 해당 관절의 각도가 운동비디오의 각도보다 크다 -> 해당 관절을 더 굽혀야 운동비디오의 각도와 같아짐
#angle differences 값이 음수이면 웹캠의 해당 관절의 각도가 운동비디오의 각도보다 작다 -> 해당 관절을 더 펴야 운동 비디오의 각도와 같아짐
#매 프레임 비교했을 시 관절이 occlusion문제로 추적이 중단 됐을때 값이 튀어서 출력이 되므로 일정 주기 동안(4초) 계속 틀렸을시 출력할 수 있도록 코드 조정

[130.12734898645274, 64.36715805655625, 140.72178585734295, 130.9151177314506, 99.31278641521226, 102.3323756844818, 165.31951033660062, 160.77439330264826]
[75.08808834760157, 30.812791215649963, 122.38892260811645, 115.1914144433223, 107.26115295136158, 129.13877043425438, 153.00438458311757, 162.08008784909015]
All angle differences: [ 55.03926064  33.55436684  18.33286325  15.72370329  -7.94836654
 -26.80639475  12.31512575  -1.30569455]
[130.98418300342755, 60.794181511863286, 137.3094386498202, 133.99491367732213, 100.93730682014323, 104.14166333811788, 164.81531395998007, 163.44312295282413]
[67.73358634009654, 31.023649019836444, 122.83685595519904, 109.26727752269083, 109.1818594638746, 135.97310965739157, 152.78838689727579, 158.17666489125497]
[132.48377811758962, 56.395641406267, 135.2415369149322, 133.70172226938587, 101.21828999570327, 105.09147715114086, 164.13461300542332, 164.15503311675295]
[71.27752332258014, 29.918974591085444, 120.40613052360898, 111.44668372379235